## This notebook will guide you through the use of the chronics you generated thorugh the Grid2op platform + display a quick analysis

<div class="alert alert-block alert-warning">
    <b>WARNING</b>: If you run <b>grid2op in a virtual env</b>, make sure that you either:
    
    - sourced your virtual env before launching jupyter: source v_env_chronix2grid/bin/activate
    
    - created an ipython kernel for this virtual env, and that you are actually running this kernerl now
</div>


In [ ]:
import os
import sys

import cufflinks as cf

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.offline
print("This notebook uses the last version of grid2op. You can install it with:\n"\
      "\t{} -m pip install grid2op".format(sys.executable))
import grid2op
if grid2op.__version__ < "0.6.0":
    raise RuntimeError("Impossible to run this notebook without grid2op version 0.6.0 installed.")
from grid2op.Chronics import ChangeNothing
from grid2op.Plot import PlotMatplotlib


# Parameters

## General parameters

In [ ]:
### CONSTANT

notebook_folder=%pwd

# define your input folder
INPUT_FOLDER = os.path.join(notebook_folder, 'example', 'input')

OUTPUT_FOLDER = os.path.join(notebook_folder, 'example', 'output')

# Detailed configuration to be set in <INPUT_FOLDER>/<CASE>/params.json
start_date = "2012-01-01"
n_scenarios = 1

CASE = 'case118_l2rpn_wcci'
path_case = os.path.join(INPUT_FOLDER, 'generation', CASE)
grid_path = os.path.join(path_case, "grid.json")

generation_output_folder=os.path.join(OUTPUT_FOLDER,'generation',CASE,start_date)#='{output_folder}/generation/{CASE}/{start_date}'

# I Create a Grid2op environment with the chronics this time

This is to test the environment can be used by grid2op. This is what the "case118_l2rpn" will look like for the competitions.


In [ ]:
generation_output_folder #='{output_folder}/generation/{CASE}/{start_date}'

In [ ]:
from grid2op.Chronics import Multifolder, GridStateFromFileWithForecasts
from grid2op.Parameters import Parameters
try:
    from lightsim2grid import LightSimBackend
    backend = LightSimBackend()
except:
    from grid2op.Backend import PandaPowerBackend
    backend = PandaPowerBackend()
    print("You might need to install the LightSimBackend to gain massive speed up")
# don't disconnect powerline on overflow, the thermal limit are not set for now, it would not make sens
param = Parameters()
param.init_from_dict({"NO_OVERFLOW_DISCONNECTION": True})

env = grid2op.make(
    "rte_case118_example",
    test=True,  
    grid_path=grid_path, # assign it the 118 grid
    chronics_class=Multifolder, # tell it to change nothing (not the most usable environment...)
    data_feeding_kwargs= {
        "path": os.path.abspath(generation_output_folder), "gridvalueClass": GridStateFromFileWithForecasts},
    param=param,
    backend=backend
)
# If you remove the "GridStateFromFileWithForecasts", from above, chronics will NOT be loaded properly.
# GridStateFromFileWithForecasts is the format used for the competition, so it is mandatory that this works!
# WITHOUT ANY MODIFICATIONS

# Beside the environment should be able to load all data generated, and not one episode.
# so please look in grid2op for compatible formats. This is not a valid format.

In [ ]:
print("Have all the chronics been loaded: {}".format(len(env.chronics_handler.real_data.subpaths) == n_scenarios))

In [ ]:
#set env thermal limit to 1 by default
th_lim = np.ones(env.n_line, dtype=np.float)
env.set_thermal_limit(th_lim)

And now we can test that we can use a Runner, store the results, and plot the flows on the powerline for example

# II Run the Grid2op Runner

For that we use a runner, that will compute the powerflows with a "do nothing" agent, and we prevent it to disconnect any power line, even if they are on overflow.

In [ ]:
from grid2op.Runner import Runner
import tempfile
from tqdm.notebook import tqdm

path_data_saved = os.path.join(os.path.abspath(os.path.join(generation_output_folder, os.pardir)), 'agent_results')#, scenario_name)
os.makedirs(path_data_saved, exist_ok=True)

nb_episode = 1#10
NB_CORE = 1#4
max_iter = 12 * 24 # one day
runner = Runner(**env.get_params_for_runner())



In [ ]:
# here you might need to change "nb_episode" if you generated more than one scenario
# this might really take some times... 4 mins per scenario per week [on one core]
res = runner.run(nb_episode=nb_episode,nb_process=NB_CORE, path_save=path_data_saved, pbar=tqdm, max_iter=max_iter)

In [ ]:
!ls $path_data_saved

In [ ]:
path_data_saved

# II Study the results
## Load the flow, prod and load data 

In [ ]:
from grid2op.Episode import EpisodeData
import numpy as np
from tqdm.notebook import tqdm

data_this_episode = EpisodeData.from_disk(path_data_saved, 'Scenario_0')

In [ ]:
path_data_saved

In [ ]:
flows_a = pd.DataFrame(np.array([obs.a_or for obs in data_this_episode.observations]))
loads_p = pd.DataFrame(np.array([obs.load_p for obs in data_this_episode.observations]))
prods_p = pd.DataFrame(np.array([obs.prod_p for obs in data_this_episode.observations]))


## Visualize energy losses & productions over time

In [ ]:
ProdTotal=prods_p.sum(axis=1)
ConsoTotal=loads_p.sum(axis=1)
Pertes=(ProdTotal-ConsoTotal)/ConsoTotal

tauxDePerte=Pertes.mean()
Pertes.describe()

In [ ]:

TotalLossesRatio=pd.DataFrame(np.array([(np.sum(obs.prod_p)-np.sum(obs.load_p))/np.sum(obs.load_p) for obs in data_this_episode.observations]))


print('average loss rate is: '+ str(TotalLossesRatio.mean()))
TotalLossesRatio.iplot(kind='scatter', filename='cufflinks/cf-simple-line')

Losses above 3.5 / 4% might be a sign that productions were not well located with respect to loads given the grid design. In real-life in such cases, you would build some new lines to reduce the losses with are usually around 2% for High Voltage grids

**Productions**

In [ ]:
prods_p.iplot(kind='scatter', filename='cufflinks/cf-simple-line')

In [ ]:
nuclear_idx = [i for i in range(len(env.gen_type)) if env.gen_type[i] == 'nuclear'] 
hydro_idx = [i for i in range(len(env.gen_type)) if env.gen_type[i] == 'hydro']
thermal_idx = [i for i in range(len(env.gen_type)) if env.gen_type[i] == 'thermal']

In [ ]:

colnames_p=prods_p.columns.values
prods_p_perType=pd.DataFrame()
prods_p_perType['nuclear']=prods_p[colnames_p[nuclear_idx]].sum(axis=1)
prods_p_perType['hydro']=prods_p[colnames_p[hydro_idx]].sum(axis=1)
prods_p_perType['thermal']=prods_p[colnames_p[thermal_idx]].sum(axis=1)
    

In [ ]:
prods_p_perType[['nuclear','hydro','thermal']].iplot(kind='scatter', filename='cufflinks/cf-simple-line')

Go back to **getting_started_api notebook** if results are not satisfying to regenerate new chronics.